**Authorize access to Google Drive**

In [1]:
from google.colab import auth
import gspread
from google.auth import default

#authenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

**Function to standardize phone number formatting**

In [2]:
import re

def standardize_phone_number(phone):
    # Remove any characters that are not digits
    phone = re.sub(r'\D', '', phone)

    # Format as 123-456-7890
    if len(phone) == 10:
        return f"{phone[:3]}-{phone[3:6]}-{phone[6:]}"
    elif len(phone) == 11 and phone.startswith('1'):
        return f"{phone[1:4]}-{phone[4:7]}-{phone[7:]}"
    else:
        return phone  # Return as is if not a 10 or 11-digit number

**Function to categorize form leads by campaign type**

In [3]:
def clean_utms(val):
    if val == 'pmax':
        return 'pmax'
    else:
        return 'search'

**Function to categorize call leads by campaign type**

In [4]:
def calls_campaign_type(row):
    if "pmax" in row['Landing Page']:
        return "pmax"
    elif "PMax" in row['Number Name']:
        return "pmax"
    else:
        return "search"

**Import, standardize, and categorize form lead data**

In [ ]:
import pandas as pd

worksheet = gc.open("Example Form Data").sheet1 #insert form leads file name between quotes

#get_all_values gives a list of rows
rows = worksheet.get_all_values()
forms = pd.DataFrame(rows)

#creating columns name
forms.columns = forms.iloc[0]
forms = forms.iloc[1:]

forms = forms[['Phone Number', 'Ad Source']]
forms['Phone Number'] = forms['Phone Number'].apply(standardize_phone_number)
forms['Forms Campaign Type'] = forms['Ad Source'].apply(clean_utms)
forms = forms.drop(labels='Ad Source', axis=1)

forms.head()

,Phone Number,Forms Campaign Type
1,555-533-0005,search
2,555-229-8428,pmax
3,555-224-9074,search
4,555-348-5270,search
5,555-413-3434,pmax


**Import, standardize, and categorize call lead data**

In [6]:
worksheet = gc.open("Example Call Data").sheet1 #insert call leads file name between quotes

#get_all_values gives a list of rows
rows = worksheet.get_all_values()
calls = pd.DataFrame(rows)

#creating columns name
calls.columns = calls.iloc[0]
calls = calls.iloc[1:]

calls['Phone Number'] = calls['Phone Number'].apply(standardize_phone_number)
calls = calls[['Phone Number', 'Number Name', 'Landing Page']]
calls['Calls Campaign Type'] = calls.apply(calls_campaign_type, axis=1)

calls.head()

,Phone Number,Number Name,Landing Page,Calls Campaign Type
1,555-312-7527,Calls from ads - PMax,,pmax
2,555-312-3378,Calls from landing page,https://www.landingpage.com?utm_source=search,search
3,555-523-6133,Calls from landing page,https://www.landingage.com?utm_source=pmax,pmax
4,555-409-6008,Calls from ads,,search
5,555-602-5141,Calls from ads - PMax,,pmax


**Merge form and call data together, and drop unnecessary columns**

In [ ]:
#drop Number Name and Landing Page
calls = calls.drop(labels=['Number Name', 'Landing Page'], axis=1)

#merge / full outer join forms and calls
leads = pd.merge(forms, calls, how='outer', on='Phone Number')
leads['Campaign Type'] = leads['Forms Campaign Type'].combine_first(leads['Calls Campaign Type'])
leads = leads.drop(labels=['Forms Campaign Type', 'Calls Campaign Type'], axis=1)
leads = leads.drop_duplicates(subset=['Phone Number'], keep='first')

leads.head()

,Phone Number,Campaign Type
0,555-533-0005,search
1,555-229-8428,pmax
2,555-224-9074,search
3,555-348-5270,search
4,555-413-3434,pmax


**Import and standardize CRM data**

In [8]:
worksheet = gc.open("Example Customer Data").sheet1 #insert CRM customer file name between quotes

#get_all_values gives a list of rows & convert to DataFrame
rows = worksheet.get_all_values()
sales = pd.DataFrame(rows)

#creating columns name
sales.columns = sales.iloc[0]
sales = sales.iloc[1:]

sales = sales[['Phone Number']]
sales['Phone Number'] = sales['Phone Number'].apply(standardize_phone_number)
sales['Sales'] = 'Sale'

sales.head()

,Phone Number,Sales
1,555-533-0005,Sale
2,555-483-4372,Sale
3,555-414-6750,Sale
4,555-670-1303,Sale
5,555-245-8671,Sale


**Merge leads and sales**

In [9]:
leads_sales = pd.merge(leads, sales, how='left', on='Phone Number')

leads_sales.head()

,Phone Number,Campaign Type,Sales
0,555-533-0005,search,Sale
1,555-229-8428,pmax,Sale
2,555-224-9074,search,NaN
3,555-348-5270,search,NaN
4,555-413-3434,pmax,NaN


**Pivot on campaign type and print results**

In [10]:
leads_sales.rename(columns={'Phone Number': 'Leads'}, inplace=True)

leads_sales = pd.pivot_table(
    leads_sales,
    index='Campaign Type',
    values=['Leads', 'Sales'],
    aggfunc='count'
)

# Get the index values instead of assuming they are integers
index_values = leads_sales.index.tolist()

# Sum all rows
if len(index_values) > 0:
    total = leads_sales.loc[index_values].sum()
    # Use pd.concat to add the total as a new row
    total_df = pd.DataFrame([total], index=['Total']) # Create DataFrame for 'total'
    leads_sales = pd.concat([leads_sales, total_df]) # Concatenate DataFrames

leads_sales['Close Rate'] = round(((leads_sales['Sales'] / leads_sales['Leads']) * 100),2)

leads_sales.head()

,Leads,Sales,Close Rate
pmax,18,4,22.22
search,26,8,30.77
Total,44,12,27.27
